In [3]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/latimeshealth.txt'
dest_url = './run_data/latimes_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [4]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su..."
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...


In [5]:
# -------------------------------------------------
frame.shape

(4171, 3)

In [6]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU'

In [7]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://lat.ms/1ELp3wU
http://lat.ms/1EKOFdr
http://lat.ms/1EHqywg
http://lat.ms/1EHqyfE
http://lat.ms/1EH3SMC
http://lat.ms/1EGNyve
http://lat.ms/1FbQ2kk
http://lat.ms/1b0E0Pi
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1wr2qv0
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1AbI1X4
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1BqdRlG
http://lat.ms/1wm7sZx
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1wm7sZx
http://lat.ms/1A84HY3
http://lat.ms/1A84HY3
http://lat.ms/1Gtx9e7
http://lat.ms/1HbYe32
http://lat.ms/1B9Gsg3
http://lat.ms/1GrwjP1
http://lat.ms/1B9Gsg3
http://lat.ms/1GrngO4
http://lat.ms/1McGw0s
http://lat.ms/1H7OGGg
http://lat.ms/1wKRkBh
http://lat.ms/1wKRkBh
http://lat.ms/1McmTr0
http://lat.ms/1EeQS0o
http://lat.ms/1Kv6yR2
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://lat.ms/17RHiCB
http://lat.ms/1M6N9

http://lati.ms/mbnev
http://lati.ms/malSM
http://lati.ms/marYs
http://lati.ms/makf6
http://lati.ms/mahro
http://lati.ms/mad0U
http://wapo.st/16FQ4OZ
http://lati.ms/m9Qvy
http://lati.ms/m9Plq
http://lati.ms/m9tEp
http://lati.ms/m9ILe
http://lati.ms/m8ZBR
http://lati.ms/m99OP
http://lati.ms/m80lg
http://lati.ms/m7ZGG
http://www.latimes.com/news/local/la-me-mta-pesticides-20130617
http://lati.ms/m7sWA
http://lati.ms/m7rOQ
http://lati.ms/m7DR6
http://instagram.com/p/apaep7n4xQ/
http://lati.ms/m70l1
http://lati.ms/m72HF
http://instagram.com/p/aqQAsbhVrR/
http://lat.ms/13LFNye
http://lati.ms/m6ujG
http://lati.ms/m5wR0
http://lati.ms/m5B9s
http://lati.ms/m55Gk
http://lati.ms/m4w9a
http://lati.ms/m32Nb
http://lati.ms/m4vfO
http://instagram.com/p/amTaYdHrGv/
http://lati.ms/m4uXz
http://instagram.com/p/akk1LwPwdd/
http://lati.ms/m3vMW
http://lati.ms/m3pzk
http://lati.ms/m3oQY
http://lati.ms/m36EJ
http://lati.ms/m3vBs
http://lati.ms/m33hr
http://lati.ms/m2V99
http://lati.ms/m3oDu
https://www.stra

http://lat.ms/wGVgck
http://lat.ms/Ax8L25
http://lat.ms/wZ1HH4
http://lat.ms/xqFsxB
http://lat.ms/wiTl0u
http://lat.ms/zphT3l
http://lat.ms/A1sK92
http://lat.ms/y2fCwA
http://lat.ms/zKRQBC
http://lat.ms/yEzmfs
http://lat.ms/A8SKre
http://lat.ms/yJwUXz
http://lat.ms/wBh2qG
http://lat.ms/wdyaDw
http://lat.ms/zYalIU
http://lat.ms/wizh0z
http://lat.ms/wEOLll
http://lat.ms/wfpE7J
http://lat.ms/ywHBlk
http://lat.ms/yPwYiX
http://lat.ms/wuNWFB
http://lat.ms/xPPchC
http://lat.ms/ArkgmD
http://lat.ms/x43gI5
http://lat.ms/zUFf4G
http://lat.ms/AizPMt
http://lat.ms/wpMg0L
http://lat.ms/yGf0O3
http://lat.ms/w0TrO0
http://lat.ms/wF5EUF
http://lat.ms/wiTsbZ
http://lat.ms/yyErzi
http://lat.ms/AucBJC
http://lat.ms/AuHzoQ
http://lat.ms/zEhrMd
http://lat.ms/zcF04Y
http://lat.ms/wkcc3j
http://lat.ms/zEAmQu
http://lat.ms/x4drxm
http://lat.ms/AsIz4s
http://lat.ms/xw01k1
http://lat.ms/yilRG2
http://lat.ms/zTxBQb
http://lat.ms/xax7Wd
http://lat.ms/ABO88J
http://lat.ms/zFAIZr
http://lat.ms/z9pmuZ
http://lat.ms

What do flu viruses and the weather have in common? Both can be unpredictable. http://lat.ms/1vttPtO


http://lat.ms/1vttPtO
Counting fingers and toes isn't enough: many parents of newborns clamor to get baby's genome sequenced. http://lat.ms/1q20TYU


http://lat.ms/1q20TYU
Designing a flu vaccine is always a guessing game. Some years, they guess better than others. via @montemorin http://lat.ms/1vttPtO


http://lat.ms/1vttPtO
Another example of how timing is everything (for your waistline). http://lat.ms/1pYVzFQ


http://lat.ms/1pYVzFQ
RT @LATerynbrown: How an app -- and bluetooth-enabled blood pressure cuffs! -- helped women improve lifestyle habits #mhealth


If you can't cut back on those tempting foods, could you at least limit your eating hours to 9 per day? http://lat.ms/1pYVzFQ


http://lat.ms/1pYVzFQ
Dominance of H3N2 flu strains (whether in the flu vaccine or not) is NOT a good sign, @DrFriedenCDC warns http://lat.ms/1FQSkCf


http://lat.ms/1FQSkCf
Message from @CDCgov to the



http://lat.ms/1lusHRk
Tosca Reno says you can ease into lunges with a little 'cheat' http://lat.ms/1liNcM9


http://lat.ms/1liNcM9
Celiac Fdn's gluten-free galore expo: Waffles, nutty bars and more - http://ow.ly/xWlSj @celiacawareness @glutenfreefind @peveteaux


http://ow.ly/xWlSj
Gluten free for dinner -- and fast! from @noellecarter http://ow.ly/xTwBV


http://ow.ly/xTwBV
Take @US_FDA &amp; @EPA's advice and eat more fish, except these: shark, swordfish, king mackerel and tilefish http://lat.ms/1oRJGw4


http://lat.ms/1oRJGw4
Want to follow @US_FDA &amp; @EPA's new advice on fish? Try cod, tilapia, salmon, shrimp, pollack and catfish: http://lat.ms/1oRJGw4


http://lat.ms/1oRJGw4
For those awesome omega-3s, @US_FDA &amp; @EPA urge women and kids to eat more fish (the kinds low in mercury): http://lat.ms/1oRJGw4


http://lat.ms/1oRJGw4
Not-so-fun fact: Money spent caring for Americans with autism each year exceeds interest paid on the national debt. http://lat.ms/1tZMRmD


http://

...

RT @renelynch: Goodnight, weekend! #latfit #10000 #omron #pedometerwars http://instagram.com/p/bxj8XhH4-N/


http://instagram.com/p/bxj8XhH4-N/
RT @mmacvean: Another gluten-free market, this one in Woodland Hills has fresh pita, cookies, cupcakes and more http://www.latimes.com/features/food/dailydish/la-dd-gluten-free-market-20130711,0,4086229.story via 


http://www.latimes.com/features/food/dailydish/la-dd-gluten-free-market-20130711
RT @MaeveReston: Last Sunday's run -- a downtown stairclimb -- inspiration for this Sunday. #LATfit http://pbs.twimg.com/media/BPGRLG5CEAEcvW1.jpg


http://pbs.twimg.com/media/BPGRLG5CEAEcvW1.jpg
Getting out and about today? Make your walk a full body exercise by vaulting over a wall – here's how: http://lati.ms/mWdj1 #LATfit


http://lati.ms/mWdj1
RT @MaeveReston: #LATfit RT @BrittanyJLadd: Morning run -- could get used to this view. #california @ Hollywood Lake http://instagram.com/p/buPvDcCitR/


http://instagram.com/p/buPvDcCitR/
Quests to be #glute

RT @latimes: Vermont governor signs 'death with dignity' measure http://lat.ms/16FRzzJ


http://lat.ms/16FRzzJ
Opposition to Medicaid expansion is likely to widen the divide between the nation's healthiest and sickest states: http://lati.ms/ldEPu


http://lati.ms/ldEPu
Gluten-free hard candy? Yep, it’s one product that’s made the gluten-free claim over the past year: http://lati.ms/ldheY


http://lati.ms/ldheY
RT @annagorman: Looking at broader community health in Oregon by @ezraklein. MT Is the future of American health care in Oregon?


When the healthcare law takes full effect, common life events could mean gaps in medical coverage for millions: http://lati.ms/ld4NL


http://lati.ms/ld4NL
DSM-5 is here, but not without already creating controversy: http://lati.ms/lcLqq What do you think of the new additions to the DSM?


http://lati.ms/lcLqq
Going gluten-free is more common, but for those with celiac disease, maintaining a strict diet still poses challenges http://lati.ms/lbkF5


ht



http://lat.ms/QxsPm7
To screen or not to screen http://lat.ms/Y4OkwC


http://lat.ms/Y4OkwC
Yosemite hantavirus outbreak has sickened 10, killed 3, CDC says http://lat.ms/TYTZ1T


http://lat.ms/TYTZ1T
People with ADHD commit more crimes while off medication, study says http://lat.ms/TYRflh


http://lat.ms/TYRflh
Smallpox virus detected in 300-year-old Siberian mummy http://lat.ms/TYRhcN


http://lat.ms/TYRhcN
Largest OB-GYN organization urges over-the-counter birth control http://lat.ms/ScUo0U


http://lat.ms/ScUo0U
Food banks collaborate to get more nutritious food to people http://lat.ms/ScUq8W


http://lat.ms/ScUq8W
Thanksgiving advice on the Web (some actually useful!) http://lat.ms/SbVwlk


http://lat.ms/SbVwlk
Testosterone supplements don't help Viagra with erections: study http://lat.ms/XWYLSP


http://lat.ms/XWYLSP
Teens want more muscle, some use steroids to get there http://lat.ms/XWYMWO


http://lat.ms/XWYMWO
Unemployment linked to increased risk of heart attacks http://la


http://lat.ms/MbRnyX
Berlin Patient, first person cured of HIV, may soon have company http://lat.ms/MbMOEB


http://lat.ms/MbMOEB
Meatless Monday flap: USDA gets into trouble with beef industry http://lat.ms/N8NiFH


http://lat.ms/N8NiFH
An abandoned weight-loss drug gets a makeover http://lat.ms/NZIc2P


http://lat.ms/NZIc2P
Squeaky clean mice show that skin bacteria help shape immunity http://lat.ms/Nx5SNr


http://lat.ms/Nx5SNr
Fetal DNA tests: Will patents work against patients? http://lat.ms/Q5U1Sq


http://lat.ms/Q5U1Sq
CT for chest pain in ER gets patients home sooner http://lat.ms/MIsxod


http://lat.ms/MIsxod
HIV drug resistance is spreading in Africa, experts say http://lat.ms/LOQaaG


http://lat.ms/LOQaaG
Exposure to light at night may contribute to depression, study says http://lat.ms/NJBU67


http://lat.ms/NJBU67
Heart attack risk balloons after surgery to fix hip or knee http://lat.ms/O8mc6Z


http://lat.ms/O8mc6Z
2012 AIDS meeting: Early treatment is key, experts say ht



http://lat.ms/yqe10C
Black Americans die up to 8 years sooner in some states http://lat.ms/AA2OdS


http://lat.ms/AA2OdS
Sex lives: Fact or fiction http://lat.ms/zl8oPp


http://lat.ms/zl8oPp
Bigger warnings on cigarette packs blocked by federal judge http://lat.ms/wpe09V


http://lat.ms/wpe09V
Sleep doesn't worsen with age, study shows http://lat.ms/zYNRZX


http://lat.ms/zYNRZX
Kids eat and drink way too much added sugar, the CDC says http://lat.ms/Anxevq


http://lat.ms/Anxevq
Bird flu research: U.S. panel may take another look http://lat.ms/xCvo4s


http://lat.ms/xCvo4s
Producing more babies via automation http://lat.ms/AzzkKU


http://lat.ms/AzzkKU
Lakers' Kobe Bryant sent for MRI post-broken nose: What does it mean? http://lat.ms/AroFnm


http://lat.ms/AroFnm
Sleeping pills linked to higher risk of cancer, death, study says http://lat.ms/xDrLy8


http://lat.ms/xDrLy8
Statins get new safety warnings http://lat.ms/wTwmRZ


http://lat.ms/wTwmRZ
End-of-life care is about much more 

In [ ]:
frame.head()

In [ ]:
frame.loc[:,'title'][0]

In [ ]:
# -------------------------------------------------
# Scrapt form web
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="entry-content"]/p[@class="story-body-text"]/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//header/div[@class="story-meta-footer"]/address/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.info()

In [ ]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [8]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
    success_date = './date/success_date.txt'
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
            f1 = open(success_date, "a")
            f1.write(frame['date'].iloc[key])
            f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  5405
1  :  2135
2  :  5918
3  :  3079
4  :  2056
5  :  2776
6  :  4668
7  :  3
[Out] key:  7
8  :  4041
9  :  3502
10  :  3502
11  :  4372
12  :  3502
13  :  3502
14  :  3502
15  :  4686
16  :  4686
17  :  7289
18  :  105
19  :  7289
20  :  4686
21  :  7289
22  :  7289
23  :  3425
24  :  3425
25  :  5737
26  :  2863
27  :  4922
28  :  4806
29  :  4922
30  :  2066
31  :  3849
32  :  2224
33  :  1681
34  :  1681
35  :  1681
36  :  3431
37  :  4922
38  :  3
[Out] key:  38
39  :  3
[Out] key:  39
40  :  3
[Out] key:  40
41  :  3
[Out] key:  41
42  :  3
[Out] key:  42
43  :  3
[Out] key:  43
44  :  3
[Out] key:  44
45  :  3
[Out] key:  45
46  :  3111
47  :  4040
48  :  3111
49  :  3431
50  :  3431
51  :  3111
52  :  3911
53  :  3911
54  :  3911
55  :  3911
56  :  3911
57  :  3911
58  :  4425
59  :  4425
60  :  4425
61  :  4425
62  :  5324
63  :  6479
64  :  4268
65  :  3170
66  :  3170
67  :  6479
68  :  5787
69  :  3129
70  :  2277
71  :  2228
72  :  2636
73  :  5186
74  :  1571
75  

731  :  4238
732  :  4558
733  :  4558
734  :  3434
735  :  4238
736  :  3434
737  :  4558
738  :  7024
739  :  2040
740  :  3532
741  :  3
[Out] key:  741
742  :  3297
743  :  3318
744  :  3521
745  :  3318
746  :  3
[Out] key:  746
747  :  4337
748  :  4337
749  :  2890
750  :  4387
751  :  4387
752  :  4387
753  :  2115
754  :  2767
755  :  2767
756  :  2767
757  :  2767
758  :  2232
759  :  2232
760  :  5835
761  :  3205
762  :  3094
763  :  4032
764  :  10177
765  :  3964
766  :  4927
767  :  3801
768  :  10177
769  :  3508
770  :  8137
771  :  3508
772  :  8137
773  :  8137
774  :  5313
775  :  5313
776  :  5728
777  :  4350
778  :  2743
779  :  4616
780  :  4039
781  :  3105
782  :  1416
783  :  3
[Out] key:  783
784  :  4540
785  :  2676
786  :  2676
787  :  4021
788  :  2876
789  :  2218
790  :  4188
791  :  3595
792  :  3474
793  :  3474
794  :  5409
795  :  2222
796  :  3180
797  :  1373
798  :  6259
799  :  4911
800  :  2240
801  :  5269
802  :  3
[Out] key:  802
803  :  40

1435  :  3
[Out] key:  1435
1436  :  2292
1437  :  2283
1438  :  1894
1439  :  2283
1440  :  4006
1441  :  2283
1442  :  2200
1443  :  4006
1444  :  3422
1445  :  2317
1446  :  3428
1447  :  3
[Out] key:  1447
1448  :  3
[Out] key:  1448
1449  :  3428
1450  :  2349
1451  :  3428
1452  :  3428
1453  :  2724
1454  :  1862
1455  :  3217
1456  :  2724
1457  :  3
[Out] key:  1457
1458  :  3
[Out] key:  1458
1459  :  3
[Out] key:  1459
1460  :  2904
1461  :  2386
1462  :  3
[Out] key:  1462
1463  :  3
[Out] key:  1463
1464  :  3044
1465  :  1738
1466  :  3286
1467  :  3044
1468  :  3
[Out] key:  1468
1469  :  3
[Out] key:  1469
1470  :  5268
1471  :  5268
1472  :  3980
1473  :  3980
1474  :  3
[Out] key:  1474
1475  :  3
[Out] key:  1475
1476  :  3
[Out] key:  1476
1477  :  3
[Out] key:  1477
1478  :  3
[Out] key:  1478
1479  :  3634
1480  :  3838
1481  :  4051
1482  :  3348
1483  :  3
[Out] key:  1483
1484  :  3
[Out] key:  1484
1485  :  5015
1486  :  3
[Out] key:  1486
1487  :  2338
1488  

1803  :  3
[Out] key:  1803
1804  :  3
[Out] key:  1804
1805  :  3
[Out] key:  1805
1806  :  3
[Out] key:  1806
1807  :  3
[Out] key:  1807
1808  :  3
[Out] key:  1808
1809  :  3
[Out] key:  1809
1810  :  3
[Out] key:  1810
1811  :  3
[Out] key:  1811
1812  :  3
[Out] key:  1812
1813  :  3
[Out] key:  1813
1814  :  3
[Out] key:  1814
1815  :  3
[Out] key:  1815
1816  :  3
[Out] key:  1816
1817  :  3
[Out] key:  1817
1818  :  3
[Out] key:  1818
1819  :  3
[Out] key:  1819
1820  :  3
[Out] key:  1820
1821  :  3
[Out] key:  1821
1822  :  3
[Out] key:  1822
1823  :  3
[Out] key:  1823
1824  :  3
[Out] key:  1824
1825  :  3
[Out] key:  1825
1826  :  3
[Out] key:  1826
1827  :  3
[Out] key:  1827
1828  :  3
[Out] key:  1828
1829  :  3
[Out] key:  1829
1830  :  3
[Out] key:  1830
1831  :  3
[Out] key:  1831
1832  :  3
[Out] key:  1832
1833  :  3
[Out] key:  1833
1834  :  3
[Out] key:  1834
1835  :  3
[Out] key:  1835
1836  :  3
[Out] key:  1836
1837  :  3
[Out] key:  1837
1838  :  3
[Out] key

[Out] key:  2103
2104  :  3
[Out] key:  2104
2105  :  3
[Out] key:  2105
2106  :  3
[Out] key:  2106
2107  :  3
[Out] key:  2107
2108  :  3
[Out] key:  2108
2109  :  3
[Out] key:  2109
2110  :  3
[Out] key:  2110
2111  :  3
[Out] key:  2111
2112  :  3
[Out] key:  2112
2113  :  3
[Out] key:  2113
2114  :  3
[Out] key:  2114
2115  :  3
[Out] key:  2115
2116  :  3
[Out] key:  2116
2117  :  3
[Out] key:  2117
2118  :  3
[Out] key:  2118
2119  :  3
[Out] key:  2119
2120  :  3
[Out] key:  2120
2121  :  3
[Out] key:  2121
2122  :  3
[Out] key:  2122
2123  :  3
[Out] key:  2123
2124  :  3
[Out] key:  2124
2125  :  3
[Out] key:  2125
2126  :  3
[Out] key:  2126
2127  :  3
[Out] key:  2127
2128  :  3
[Out] key:  2128
2129  :  3
[Out] key:  2129
2130  :  3
[Out] key:  2130
2131  :  3
[Out] key:  2131
2132  :  3
[Out] key:  2132
2133  :  3
[Out] key:  2133
2134  :  3
[Out] key:  2134
2135  :  3
[Out] key:  2135
2136  :  3
[Out] key:  2136
2137  :  3
[Out] key:  2137
2138  :  3
[Out] key:  2138
213

...

[Out] key:  2705
2706  :  3
[Out] key:  2706
2707  :  3
[Out] key:  2707
2708  :  3
[Out] key:  2708
2709  :  4168
2710  :  1741
2711  :  3
[Out] key:  2711
2712  :  4050
2713  :  1638
2714  :  3
[Out] key:  2714
2715  :  3
[Out] key:  2715
2716  :  3
[Out] key:  2716
2717  :  3
[Out] key:  2717
2718  :  3
[Out] key:  2718
2719  :  3
[Out] key:  2719
2720  :  3
[Out] key:  2720
2721  :  3
[Out] key:  2721
2722  :  3
[Out] key:  2722
2723  :  3
[Out] key:  2723
2724  :  3
[Out] key:  2724
2725  :  3
[Out] key:  2725
2726  :  3
[Out] key:  2726
2727  :  1156
2728  :  3
[Out] key:  2728
2729  :  3
[Out] key:  2729
2730  :  3
[Out] key:  2730
2731  :  3
[Out] key:  2731
2732  :  3
[Out] key:  2732
2733  :  3
[Out] key:  2733
2734  :  3
[Out] key:  2734
2735  :  3
[Out] key:  2735
2736  :  3
[Out] key:  2736
2737  :  3
[Out] key:  2737
2738  :  3
[Out] key:  2738
2739  :  3
[Out] key:  2739
2740  :  3
[Out] key:  2740
2741  :  3
[Out] key:  2741
2742  :  3
[Out] key:  2742
2743  :  2897
274

3228  :  3
[Out] key:  3228
3229  :  3
[Out] key:  3229
3230  :  3
[Out] key:  3230
3231  :  3
[Out] key:  3231
3232  :  3
[Out] key:  3232
3233  :  3
[Out] key:  3233
3234  :  3
[Out] key:  3234
3235  :  3
[Out] key:  3235
3236  :  3
[Out] key:  3236
3237  :  3
[Out] key:  3237
3238  :  3
[Out] key:  3238
3239  :  3
[Out] key:  3239
3240  :  3
[Out] key:  3240
3241  :  3
[Out] key:  3241
3242  :  3
[Out] key:  3242
3243  :  3
[Out] key:  3243
3244  :  3
[Out] key:  3244
3245  :  3
[Out] key:  3245
3246  :  3
[Out] key:  3246
3247  :  3
[Out] key:  3247
3248  :  3
[Out] key:  3248
3249  :  3
[Out] key:  3249
3250  :  3
[Out] key:  3250
3251  :  3
[Out] key:  3251
3252  :  3
[Out] key:  3252
3253  :  3
[Out] key:  3253
3254  :  3
[Out] key:  3254
3255  :  3
[Out] key:  3255
3256  :  3
[Out] key:  3256
3257  :  3
[Out] key:  3257
3258  :  3
[Out] key:  3258
3259  :  3
[Out] key:  3259
3260  :  3
[Out] key:  3260
3261  :  3
[Out] key:  3261
3262  :  3
[Out] key:  3262
3263  :  3
[Out] key

3530  :  3
[Out] key:  3530
3531  :  3
[Out] key:  3531
3532  :  3
[Out] key:  3532
3533  :  3
[Out] key:  3533
3534  :  3
[Out] key:  3534
3535  :  3
[Out] key:  3535
3536  :  3
[Out] key:  3536
3537  :  3
[Out] key:  3537
3538  :  3
[Out] key:  3538
3539  :  3
[Out] key:  3539
3540  :  3
[Out] key:  3540
3541  :  3
[Out] key:  3541
3542  :  3
[Out] key:  3542
3543  :  3
[Out] key:  3543
3544  :  3
[Out] key:  3544
3545  :  3
[Out] key:  3545
3546  :  3
[Out] key:  3546
3547  :  3
[Out] key:  3547
3548  :  3
[Out] key:  3548
3549  :  3
[Out] key:  3549
3550  :  3
[Out] key:  3550
3551  :  3
[Out] key:  3551
3552  :  3
[Out] key:  3552
3553  :  3
[Out] key:  3553
3554  :  3
[Out] key:  3554
3555  :  3
[Out] key:  3555
3556  :  3
[Out] key:  3556
3557  :  3551
3558  :  3
[Out] key:  3558
3559  :  3
[Out] key:  3559
3560  :  432
3561  :  2199
3562  :  3380
3563  :  3
[Out] key:  3563
3564  :  3307
3565  :  3128
3566  :  3
[Out] key:  3566
3567  :  3
[Out] key:  3567
3568  :  3
[Out] key:

[Out] key:  3851
3852  :  3
[Out] key:  3852
3853  :  3
[Out] key:  3853
3854  :  5711
3855  :  3
[Out] key:  3855
3856  :  3
[Out] key:  3856
3857  :  3
[Out] key:  3857
3858  :  3
[Out] key:  3858
3859  :  3
[Out] key:  3859
3860  :  3
[Out] key:  3860
3861  :  3
[Out] key:  3861
3862  :  3
[Out] key:  3862
3863  :  3
[Out] key:  3863
3864  :  3
[Out] key:  3864
3865  :  3
[Out] key:  3865
3866  :  3
[Out] key:  3866
3867  :  3
[Out] key:  3867
3868  :  3
[Out] key:  3868
3869  :  3
[Out] key:  3869
3870  :  3
[Out] key:  3870
3871  :  3
[Out] key:  3871
3872  :  3
[Out] key:  3872
3873  :  3
[Out] key:  3873
3874  :  3
[Out] key:  3874
3875  :  3
[Out] key:  3875
3876  :  3
[Out] key:  3876
3877  :  3
[Out] key:  3877
3878  :  3
[Out] key:  3878
3879  :  3
[Out] key:  3879
3880  :  3
[Out] key:  3880
3881  :  3
[Out] key:  3881
3882  :  3
[Out] key:  3882
3883  :  3
[Out] key:  3883
3884  :  3
[Out] key:  3884
3885  :  3
[Out] key:  3885
3886  :  3
[Out] key:  3886
3887  :  3
[Out] 

4157  :  3406
4158  :  3
[Out] key:  4158
4159  :  3
[Out] key:  4159
4160  :  3
[Out] key:  4160
4161  :  3
[Out] key:  4161
4162  :  3
[Out] key:  4162
4163  :  3
[Out] key:  4163
4164  :  3
[Out] key:  4164
4165  :  3
[Out] key:  4165
4166  :  2845
4167  :  3
[Out] key:  4167
4168  :  3
[Out] key:  4168
4169  :  3
[Out] key:  4169
4170  :  3
[Out] key:  4170


In [9]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  30
1  :  30
2  :  30
3  :  30
4  :  30
5  :  30
6  :  30
7  :  30
8  :  30
9  :  30
10  :  30
11  :  30
12  :  30
13  :  30
14  :  30
15  :  30
16  :  30
17  :  30
18  :  30
19  :  30
20  :  30
21  :  30
22  :  30
23  :  30
24  :  30
25  :  30
26  :  30
27  :  30
28  :  30
29  :  30
30  :  30
31  :  30
32  :  30
33  :  30
34  :  30
35  :  30
36  :  30
37  :  30
38  :  30
39  :  30
40  :  30
41  :  30
42  :  30
43  :  30
44  :  30
45  :  30
46  :  30
47  :  30
48  :  30
49  :  30
50  :  30
51  :  30
52  :  30
53  :  30
54  :  30
55  :  30
56  :  30
57  :  30
58  :  30
59  :  30
60  :  30
61  :  30
62  :  30
63  :  30
64  :  30
65  :  30
66  :  30
67  :  30
68  :  30
69  :  30
70  :  30
71  :  30
72  :  30
73  :  30
74  :  30
75  :  30
76  :  30
77  :  30
78  :  30
79  :  30
80  :  30
81  :  30
82  :  30
83  :  30
84  :  30
85  :  30
86  :  30
87  :  30
88  :  30
89  :  30
90  :  30
91  :  30
92  :  30
93  :  30
94  :  30
95  :  30
96  :  30
97  :  30
98  :  30
99  :  30
100  :  30

847  :  30
848  :  30
849  :  30
850  :  30
851  :  30
852  :  30
853  :  30
854  :  30
855  :  30
856  :  30
857  :  30
858  :  30
859  :  30
860  :  30
861  :  30
862  :  30
863  :  30
864  :  30
865  :  30
866  :  30
867  :  30
868  :  30
869  :  30
870  :  30
871  :  30
872  :  30
873  :  30
874  :  30
875  :  30
876  :  30
877  :  30
878  :  30
879  :  30
880  :  30
881  :  30
882  :  30
883  :  30
884  :  30
885  :  30
886  :  30
887  :  30
888  :  30
889  :  30
890  :  30
891  :  30
892  :  30
893  :  30
894  :  30
895  :  30
896  :  30
897  :  30
898  :  30
899  :  30
900  :  30
901  :  30
902  :  30
903  :  30
904  :  30
905  :  30
906  :  30
907  :  30
908  :  30
909  :  30
910  :  30
911  :  30
912  :  30
913  :  30
914  :  30
915  :  30
916  :  30
917  :  30
918  :  30
919  :  30
920  :  30
921  :  30
922  :  30
923  :  30
924  :  30
925  :  30
926  :  30
927  :  30
928  :  30
929  :  30
930  :  30
931  :  30
932  :  30
933  :  30
934  :  30
935  :  30
936  :  30
937  :  30

1557  :  30
1558  :  30
1559  :  30
1560  :  30
1561  :  30
1562  :  30
1563  :  30
1564  :  30
1565  :  30
1566  :  30
1567  :  30
1568  :  30
1569  :  30
1570  :  30
1571  :  30
1572  :  30
1573  :  30
1574  :  30
1575  :  30
1576  :  30
1577  :  30
1578  :  30
1579  :  30
1580  :  30
1581  :  30
1582  :  30
1583  :  30
1584  :  30
1585  :  30
1586  :  30
1587  :  30
1588  :  30
1589  :  30
1590  :  30
1591  :  30
1592  :  30
1593  :  30
1594  :  30
1595  :  30
1596  :  30
1597  :  30
1598  :  30
1599  :  30
1600  :  30
1601  :  30
1602  :  30
1603  :  30
1604  :  30
1605  :  30
1606  :  30
1607  :  30
1608  :  30
1609  :  30
1610  :  30
1611  :  30
1612  :  30
1613  :  30
1614  :  30
1615  :  30
1616  :  30
1617  :  30
1618  :  30
1619  :  30
1620  :  30
1621  :  30
1622  :  30
1623  :  30
1624  :  30
1625  :  30
1626  :  30
1627  :  30
1628  :  30
1629  :  30
1630  :  30
1631  :  30
1632  :  30
1633  :  30
1634  :  30
1635  :  30
1636  :  30
1637  :  30
1638  :  30
1639  :  30
1640

2261  :  30
2262  :  30
2263  :  30
2264  :  30
2265  :  30
2266  :  30
2267  :  30
2268  :  30
2269  :  30
2270  :  30
2271  :  30
2272  :  30
2273  :  30
2274  :  30
2275  :  30
2276  :  30
2277  :  30
2278  :  30
2279  :  30
2280  :  30
2281  :  30
2282  :  30
2283  :  30
2284  :  30
2285  :  30
2286  :  30
2287  :  30
2288  :  30
2289  :  30
2290  :  30
2291  :  30
2292  :  30
2293  :  30
2294  :  30
2295  :  30
2296  :  30
2297  :  30
2298  :  30
2299  :  30
2300  :  30
2301  :  30
2302  :  30
2303  :  30
2304  :  30
2305  :  30
2306  :  30
2307  :  30
2308  :  30
2309  :  30
2310  :  30
2311  :  30
2312  :  30
2313  :  30
2314  :  30
2315  :  30
2316  :  30
2317  :  30
2318  :  30
2319  :  30
2320  :  30
2321  :  30
2322  :  30
2323  :  30
2324  :  30
2325  :  30
2326  :  30
2327  :  30
2328  :  30
2329  :  30
2330  :  30
2331  :  30
2332  :  30
2333  :  30
2334  :  30
2335  :  30
2336  :  30
2337  :  30
2338  :  30
2339  :  30
2340  :  30
2341  :  30
2342  :  30
2343  :  30
2344

2963  :  30
2964  :  30
2965  :  30
2966  :  30
2967  :  30
2968  :  30
2969  :  30
2970  :  30
2971  :  30
2972  :  30
2973  :  30
2974  :  30
2975  :  30
2976  :  30
2977  :  30
2978  :  30
2979  :  30
2980  :  30
2981  :  30
2982  :  30
2983  :  30
2984  :  30
2985  :  30
2986  :  30
2987  :  30
2988  :  30
2989  :  30
2990  :  30
2991  :  30
2992  :  30
2993  :  30
2994  :  30
2995  :  30
2996  :  30
2997  :  30
2998  :  30
2999  :  30
3000  :  30
3001  :  30
3002  :  30
3003  :  30
3004  :  30
3005  :  30
3006  :  30
3007  :  30
3008  :  30
3009  :  30
3010  :  30
3011  :  30
3012  :  30
3013  :  30
3014  :  30
3015  :  30
3016  :  30
3017  :  30
3018  :  30
3019  :  30
3020  :  30
3021  :  30
3022  :  30
3023  :  30
3024  :  30
3025  :  30
3026  :  30
3027  :  30
3028  :  30
3029  :  30
3030  :  30
3031  :  30
3032  :  30
3033  :  30
3034  :  30
3035  :  30
3036  :  30
3037  :  30
3038  :  30
3039  :  30
3040  :  30
3041  :  30
3042  :  30
3043  :  30
3044  :  30
3045  :  30
3046

3653  :  30
3654  :  30
3655  :  30
3656  :  30
3657  :  30
3658  :  30
3659  :  30
3660  :  30
3661  :  30
3662  :  30
3663  :  30
3664  :  30
3665  :  30
3666  :  30
3667  :  30
3668  :  30
3669  :  30
3670  :  30
3671  :  30
3672  :  30
3673  :  30
3674  :  30
3675  :  30
3676  :  30
3677  :  30
3678  :  30
3679  :  30
3680  :  30
3681  :  30
3682  :  30
3683  :  30
3684  :  30
3685  :  30
3686  :  30
3687  :  30
3688  :  30
3689  :  30
3690  :  30
3691  :  30
3692  :  30
3693  :  30
3694  :  30
3695  :  30
3696  :  30
3697  :  30
3698  :  30
3699  :  30
3700  :  30
3701  :  30
3702  :  30
3703  :  30
3704  :  30
3705  :  30
3706  :  30
3707  :  30
3708  :  30
3709  :  30
3710  :  30
3711  :  30
3712  :  30
3713  :  30
3714  :  30
3715  :  30
3716  :  30
3717  :  30
3718  :  30
3719  :  30
3720  :  30
3721  :  30
3722  :  30
3723  :  30
3724  :  30
3725  :  30
3726  :  30
3727  :  30
3728  :  30
3729  :  30
3730  :  30
3731  :  30
3732  :  30
3733  :  30
3734  :  30
3735  :  30
3736